# Imports

In [1]:
import os
import cv2
import glob
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier


from keras.models import Sequential
import keras

2022-12-14 16:10:44.595915: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 16:10:44.780617: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-14 16:10:44.844892: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-14 16:10:45.520061: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Get data

data is already augmented

In [3]:
data_dir = os.path.join("..", "yolo_preperation", "data")

In [7]:
def import_yolo_data(data_dir, split, stride=1):
    image_path = os.path.join(data_dir, "images", split)
    label_path = os.path.join(data_dir, "labels", split)
    
    images = []
    labels = []
    
    i = 0
    
    for file_name in glob.glob(os.path.join(image_path, "*.jpg")):
        if i % stride == 0:
            img = cv2.imread(file_name)
            height, width = img.shape[:2]
            images.append(cv2.resize(img, (int(width / 2), int(height / 2))))
        i += 1
        
        
    i = 0
    
    for label_txt in glob.glob(os.path.join(label_path, "*.txt")):
        if i % stride == 0:
            with open(label_txt) as f:
                labels.append(int(f.readlines()[0].split(" ")[0]))
        i += 1
            
    return (images, labels)

In [8]:
stride = 16
trian_X, train_y = import_yolo_data(data_dir, "train", stride)
val_X, val_y = import_yolo_data(data_dir, "val", stride)
train_X, train_y = import_yolo_data(data_dir, "test", stride)

print(len(train_X))

100


In [9]:
train_y = OneHotEncoder().fit_transform(np.array(train_y).reshape(-1,1)).reshape(-1, 4).toarray()

# model creation

In [14]:

# # Initialising the CNN
# classifier = Sequential()

# # Step 1 - Convolution
# classifier.add(Convolution2D(32, (3, 3), input_shape = (720, 128, 3), activation = 'relu'))

# # Step 2 - Pooling
# classifier.add(MaxPooling2D(pool_size = (2, 2)))

# # Adding a second convolutional layer
# classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))
# classifier.add(MaxPooling2D(pool_size = (2, 2)))

# # Adding a third convolutional layer
# classifier.add(Convolution2D(64, (3, 3), activation = 'relu'))          # 64 feature detectors, can be changed to something else
# classifier.add(MaxPooling2D(pool_size = (2, 2)))

# # Adding a fourth convolutional layer
# classifier.add(Convolution2D(64, (3, 3), activation = 'relu'))          # 64 feature detectors, can be changed to something else
# classifier.add(MaxPooling2D(pool_size = (2, 2)))

# # Step 3 - Flattening
# classifier.add(Flatten())

# # Step 4 - Full connection
# classifier.add(Dense(units = 128, activation = 'relu', use_bias=True, bias_initializer='ones'))
# classifier.add(Dropout(0.5))
# classifier.add(Dense(units = 5, activation = 'softmax', use_bias=True, bias_initializer='ones'))

# # Compiling the CNN
# classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


def create_model(dropout_rate=0.0):
    model = Sequential([
    keras.layers.Conv2D(32, (3, 3), input_shape = (360, 640, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation = 'relu', use_bias=True, bias_initializer='ones'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4, activation = 'softmax', use_bias=True, bias_initializer='ones'),
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(model=create_model, dropout_rate=0.2)


In [15]:
parameters = dict(epochs=[10,20,30])

In [16]:
clf = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=1, cv=3, scoring="accuracy")

In [17]:
grid_result = clf.fit(train_X, train_y)

Epoch 1/10
3/3 [==============================] - 2s 367ms/step - loss: 97.6499 - accuracy: 0.3485
Epoch 2/10
3/3 [==============================] - 1s 362ms/step - loss: 22.8523 - accuracy: 0.2424
Epoch 3/10
3/3 [==============================] - 1s 360ms/step - loss: 8.9997 - accuracy: 0.2727
Epoch 4/10
3/3 [==============================] - 1s 372ms/step - loss: 2.8077 - accuracy: 0.2879
Epoch 5/10
3/3 [==============================] - 1s 365ms/step - loss: 0.8821 - accuracy: 0.4394
Epoch 6/10
3/3 [==============================] - 1s 354ms/step - loss: 0.5443 - accuracy: 0.6061
Epoch 7/10
3/3 [==============================] - 1s 385ms/step - loss: 2.5986 - accuracy: 0.3333
Epoch 8/10
3/3 [==============================] - 1s 383ms/step - loss: 0.7475 - accuracy: 0.5000
Epoch 9/10
3/3 [==============================] - 1s 369ms/step - loss: 0.5192 - accuracy: 0.5606
Epoch 10/10
2/2 [==============================] - 0s 17ms/step
Epoch 1/10
3/3 [==============================] - 2s

3/3 [==============================] - 1s 369ms/step - loss: 2.1296 - accuracy: 0.2239
Epoch 5/20
3/3 [==============================] - 1s 376ms/step - loss: 1.0807 - accuracy: 0.2687
Epoch 6/20
3/3 [==============================] - 1s 378ms/step - loss: 0.8015 - accuracy: 0.2537
Epoch 7/20
3/3 [==============================] - 1s 391ms/step - loss: 0.6953 - accuracy: 0.3134
Epoch 8/20
3/3 [==============================] - 1s 380ms/step - loss: 0.6689 - accuracy: 0.2388
Epoch 9/20
3/3 [==============================] - 1s 392ms/step - loss: 0.6362 - accuracy: 0.3134
Epoch 10/20
3/3 [==============================] - 1s 391ms/step - loss: 0.6199 - accuracy: 0.3284
Epoch 11/20
3/3 [==============================] - 1s 375ms/step - loss: 0.6744 - accuracy: 0.2687
Epoch 12/20
3/3 [==============================] - 1s 377ms/step - loss: 0.6842 - accuracy: 0.2537
Epoch 13/20
3/3 [==============================] - 1s 383ms/step - loss: 0.5993 - accuracy: 0.4030
Epoch 14/20
3/3 [==========

Epoch 29/30
3/3 [==============================] - 2s 405ms/step - loss: 0.0897 - accuracy: 0.9552
Epoch 30/30
2/2 [==============================] - 0s 11ms/step
Epoch 1/30
3/3 [==============================] - 2s 432ms/step - loss: 55.9638 - accuracy: 0.2687
Epoch 2/30
3/3 [==============================] - 1s 395ms/step - loss: 16.9754 - accuracy: 0.2537
Epoch 3/30
3/3 [==============================] - 2s 393ms/step - loss: 4.7767 - accuracy: 0.2388
Epoch 4/30
3/3 [==============================] - 1s 382ms/step - loss: 0.9878 - accuracy: 0.3582
Epoch 5/30
3/3 [==============================] - 1s 381ms/step - loss: 0.7709 - accuracy: 0.3731
Epoch 6/30
3/3 [==============================] - 1s 392ms/step - loss: 0.6728 - accuracy: 0.3284
Epoch 7/30
3/3 [==============================] - 1s 382ms/step - loss: 0.4893 - accuracy: 0.6119
Epoch 8/30
3/3 [==============================] - 1s 389ms/step - loss: 0.5284 - accuracy: 0.7015
Epoch 9/30
3/3 [==============================] - 2

In [18]:
grid_result

GridSearchCV(cv=3,
             estimator=KerasClassifier(dropout_rate=0.2, model=<function create_model at 0x7f57300432e0>),
             n_jobs=1, param_grid={'epochs': [10, 20, 30]}, scoring='accuracy')